In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [11]:
url = "https://boston.craigslist.org/search/sof"

response = requests.get(url)
response

<Response [200]>

In [3]:
data = response.text

In [4]:
data

'\ufeff<!DOCTYPE html>\n<html class="no-js"><head>\n    <title>boston software/qa/dba/etc  - craigslist</title>\n\n    <meta name="description" content="boston software/qa/dba/etc  - craigslist">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n    <link rel="canonical" href="https://boston.craigslist.org/search/sof">\n    <link rel="alternate" type="application/rss+xml" href="https://boston.craigslist.org/search/sof?format=rss" title="RSS feed for craigslist | boston software/qa/dba/etc  - craigslist">\n    <meta name="viewport" content="width=device-width,initial-scale=1">\n    <link type="text/css" rel="stylesheet" media="all" href="//www.craigslist.org/styles/cl.css?v=281764e2707bd58e05233e4b7df36df8">\n    <link type="text/css" rel="stylesheet" media="all" href="//www.craigslist.org/styles/search.css?v=bc035cbbc3978b0ec9df93944cdf349b">\n    <link type="text/css" rel="stylesheet" media="all" href="//www.craigslist.org/styles/jquery-ui-clcustom.css?v=3b05ddffb7c7f5b6206

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
tags = soup.find_all('a')

In [8]:
titles = soup.find_all('a',{'class':'result-title'})

for title in titles:
    print(title.text)

looking for .net web developer
Executive Assistant Supporting CEO and CFO
Data Analyst / Data Engineer
Software Engineering Fellowship
Growth Stage Edtech Startup Hiring Full Stack Engineers
Information Security Analyst
Mid-Level Software Engineer
Clinical Content Developer
Machine learning for Design Tools
Order Management Specialist
ADS UI/UX IT Specialist
Principal Software Engineer
Senior Software Engineer
SWE for Stealth Start-Up
iOS Mobile Developer
Senior Telecommunications Engineer
Customer Success Manager
Test Automation Engineer
Senior Software Engineer
Mobile Engineer
Entry Level Software Tester
Technical Consultant / Software Developer
System Administration and Security Analyst
Director, Software Quality Assurance and Release Engineering
Senior Project Manager for Software Company
iOS/Android Mobile Developer
Software Engineering Fellowship
Growth Stage Edtech Startup Hiring Full Stack Engineers
Cloud Solutions Engineer
Autonomous Vehicle Operator | Toyota Research Institut

In [7]:
for tag in tags:
    print(tag.get('href'))

/
/
https://post.craigslist.org/c/bos
https://accounts.craigslist.org/login/home
#
#
/
https://accounts.craigslist.org/login?rt=L&rp=%2Fsavesearch%2Fsave%3FURL%3Dhttps%253A%252F%252Fboston%252Ecraigslist%252Eorg%252Fsearch%252Fsof
/search/sof
/search/sof
/search/sof?sort=date&
/search/sof
/search/sof

https://boston.craigslist.org/gbs/sof/d/maynard-looking-for-net-web-developer/6952655771.html
https://boston.craigslist.org/gbs/sof/d/maynard-looking-for-net-web-developer/6952655771.html
#
https://boston.craigslist.org/gbs/sof/d/cambridge-executive-assistant/6952228202.html
https://boston.craigslist.org/gbs/sof/d/cambridge-executive-assistant/6952228202.html
#
https://boston.craigslist.org/nwb/sof/d/westford-data-analyst-data-engineer/6951784732.html
https://boston.craigslist.org/nwb/sof/d/westford-data-analyst-data-engineer/6951784732.html
#
https://boston.craigslist.org/gbs/sof/d/boston-software-engineering-fellowship/6951764302.html
https://boston.craigslist.org/gbs/sof/d/boston-softw

In [16]:
adrss = soup.find_all('span',{'class':'result-hood'})

for adrs in adrss:
    print(adrs.text)

 (Maynard)
 (Cambridge, MA)
 (westford)
 (Cambridge MA)
 (Boston)
 (Quincy, MA)
 (Newton MA)
 (Cambridge)
 (Cambridge, MA)
 (Cambridge, MA)
 (Boston)
 (Chestnut Hill)
 (Walham)
 (Boston/Cambridge/Brookline)
 (Boston/Cambridge/Brookline)
 (Boston)
 (Boston/Cambridge/Brookline)
 (Nahsua NH)
 (Cambridge)
 (Boston (Remote))
 (Walham)
 (Boston, Massachusetts)
 (Cambridge MA)
 (Walham)
 (cambridge)
 (Boston, MA)
 (Cambridge)
 (boston)
 (BRAINTREE)
 (Quincy, MA)
 (Boston, MA)
 (cambridge)
 (Medford)
 (Acton, MA)
 (cambridge)


In [14]:
scre_txt = soup.find_all('span', {'class':'screen-reader-text'})

In [28]:
job_number = 0
bosten_jobs = {}
while True:
    url = "https://boston.craigslist.org/search/sof"
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    jobs = soup.find_all('p',{'class':'result-info'})


    for job in jobs:
        title = job.find('a', {'class':'result-title'}).text

        link = job.find('a',{'class':'result-title'}).get('href')

        dte_tag = job.find('time', {'class':'result-date'})
        dte = dte_tag.text if dte_tag else "post date not present"

        adrs_tag = job.find('span', {'class':'result-hood'})
        adrs = adrs_tag.text[2:-1] if adrs_tag else "No location shared"

        link_response = requests.get(link)
        job_data = link_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('section', {'id':'postingbody'}).text
        job_attribute_tag = job_soup.find('p',{'class':'attrgroup'})
        job_attribute = job_attribute_tag.text if job_attribute_tag else 'No job attribute given'
        
        job_number +=1
        bosten_jobs[job_number] = [title, adrs, dte, link, job_attribute, job_description]
        
    url_tg = soup.find('a',{'title':'next page'})
    if url_tg.get('href'):
        url = 'https://boston.craigslist.org' + url_tg.get('href')
    else:
        break

col_names = ['Job title', 'location', 'date posted', 'link of job', 'job attribute', 'job description']
bosten_jobs_df = pd.DataFrame.from_dict(bosten_jobs, orient = 'index', columns = col_names)    

In [29]:
bosten_jobs_df.head()

,Job title,location,date posted,link of job,job attribute,job description
1,looking for .net web developer,Maynard,Aug 8,https://boston.craigslist.org/gbs/sof/d/maynar...,\ncompensation: 90 - 120\n\nemployment type: f...,\n\nQR Code Link to This Post\n\n\nJoin the I....
2,Executive Assistant Supporting CEO and CFO,"Cambridge, MA",Aug 8,https://boston.craigslist.org/gbs/sof/d/cambri...,\ncompensation: $35-$45/hr.\n\nemployment type...,\n\nQR Code Link to This Post\n\n\nExecutive A...
3,Data Analyst / Data Engineer,westford,Aug 7,https://boston.craigslist.org/nwb/sof/d/westfo...,\ncompensation: Per Hour Pay ($10 - $30)\n\nem...,\n\nQR Code Link to This Post\n\n\nStartup loo...
4,Software Engineering Fellowship,No location shared,Aug 7,https://boston.craigslist.org/gbs/sof/d/boston...,"\ncompensation: 80,000-160,000\n\nemployment t...",\n\nQR Code Link to This Post\n\n\nWhat is Pat...
5,Growth Stage Edtech Startup Hiring Full Stack ...,Cambridge MA,Aug 7,https://boston.craigslist.org/gbs/sof/d/cambri...,"\ncompensation: Competitive with Market, Comme...",\n\nQR Code Link to This Post\n\n\nPrepScholar...


In [30]:
bosten_jobs_df.to_csv('bosten_jobs.csv')